# P 12.1

a) Implement Gram-Schmidt reduction algorithm on SAGE.

In [1]:
def proj(u, v):
    return (u.inner_product(v) / u.inner_product(u)) * u

def Gram_Schmidt_reduction(basis):
    """ Compute the orthogonal basis of a given basis using the Gram-Schmidt algorithm. """
    
    # Initialize the orthogonal basis
    orth_basis = [basis[0]]

    # Compute the orthogonal basis using the Gram-Schmidt updating process
    for i in range(1, len(basis)):
        temp = vector([0]*len(basis))
        for j in range(i):
            temp = temp + proj(orth_basis[j], basis[i])
        orth_basis.append(basis[i] - temp)

    return orth_basis

b) Implement the LLL algorithm on SAGE as seen in class.

In [2]:
def my_LLL(basis, delta = 3/4): 
    """ my_LLL function must be called with argument being `copy(basis)` in order to compare it to the pre-implemented LLL method from SageMath. """
    
    # Setup
    mu = {}
    n = len(basis)

    # Gram-Schmidt orthogonalization
    basis_hat = Gram_Schmidt_reduction(basis)
    
    # Reduction stage
    for i in range(2, n+1):
        for j in reversed(range(1, i)):
            mu[(i,j)] = basis[i-1].inner_product(basis_hat[j-1]) / basis_hat[j-1].inner_product(basis_hat[j-1])
            basis[i-1] = basis[i-1] - round(mu[(i,j)])*basis[j-1]
            mu[(i,j)] = basis[i-1].inner_product(basis_hat[j-1]) / basis_hat[j-1].inner_product(basis_hat[j-1]) # Algorithm in the notes has this implicit update
   
    # Swap stage
    for i in range(1, n):
        if delta*(basis_hat[i-1].norm()**2) > (mu[(i+1,i)]*basis_hat[i-1] + basis_hat[i]).norm()**2:
            # Swap b_i and b_{i+1}
            temp = basis[i-1]
            basis[i-1] = basis[i]
            basis[i] = temp

            # Go to step 1
            my_LLL(basis, delta)
    return basis

Test it on the lattices with $\mathbb Z$-basis [(512,1024),(271,512)] and [(314,159,265),(-27,18.28),(0,1,7)] respectively (and compare your results with SAGE pre-implemented LLL function).

In [5]:
""" Note: SageMath's pre-implemented LLL method returns LLL-reduced matrix of the lattice generated by the rows of self. """

# Define the parameter for the algorithm
delta = 3/4

# Check n.1
basis1 = [vector([512, 1024]), vector([271, 512])]
print(f"Does the newly implemented function for the LLL algorithm work as the SageMath pre-implemented method for basis?\n>> {matrix(my_LLL(copy(basis1))) == matrix(basis1).LLL(delta)}")

# Check n.2
basis2 = [vector([314, 159, 265]), vector([-27, 18, 28]), vector([0, 1, 7])]
print(f"\nDoes the newly implemented function for the LLL algorithm work as the SageMath pre-implemented method for basis?\n>> {matrix(my_LLL(copy(basis2))) == matrix(basis2).LLL(delta)}")
assert matrix(my_LLL(copy(basis2))) == matrix(basis2).LLL(delta)

Does the newly implemented function for the LLL algorithm work as the SageMath pre-implemented method for basis?
>> True

Does the newly implemented function for the LLL algorithm work as the SageMath pre-implemented method for basis?
>> True


# P 12.2

As seen in class, we can apply lattice reduction algorithms (as LLL) to a question in arithmetic.
Given a prime number $p \equiv 1 \mod 4$, it is known that there are integers $a, b \in \mathbb Z$ such that $p = a^2 + b^2$ (Fermat’s two squares theorem).

Find $\alpha \in \mathbb Z$ such that $\alpha^2 \equiv -1 \mod p$ and then consider the lattice $L = \langle (p,0), (\alpha,1) \rangle$.

By running the LLL algorithm on $L$, find integers $a,b$ such that $p = a^2 + b^2$, where $p = 10^{100} + 949$.

In [4]:
from sage.rings.finite_rings.integer_mod import square_root_mod_prime_power

# Define the prime number p and check if it is congruent to 1 modulo 4
p = 10**100 + 949
assert p % 4 == 1

# Find a square root of -1 modulo p
a = Mod(-1, p)
alpha = square_root_mod_prime_power(a, p, 1)

# By running the LLL algorithm on L generated by [v1, v2], find integers a,b such that p = a^2 + b^2
v1 = vector([int(p), 0])
v2 = vector([int(alpha),1])
basis3 = [v1, v2]

# Run the LLL algorithm
L = matrix(basis3).LLL()    # LLL method returns LLL-reduced matrix of the lattice generated by the rows of self.

# Extract a and b
a = L[0][0]
b = L[1][0]

# Return the values of a and b
print(f"a = {a}")
print(f"b = {b}")

# Check if p = a^2 + b^2
print("\np == a^2 + b^2 (mod p) ? \n>>", p == a**2 + b**2)

a = -99697921470138519447541656418848509184628524016382
b = -7766881905507050845172598218029833369440123277895

p == a^2 + b^2 (mod p) ? 
>> True
